<a href="https://colab.research.google.com/github/eunShim/BigDataCapstone/blob/main/bigdatacapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!sudo apt-get install -y p7zip-full
!mkdir -p /content/KT4_extracted
!7z x /content/EdNet-KT4.zip -o/content/KT4_extracted



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 175112192 bytes (167 MiB)

Extracting archive: /content/EdNet-KT4.zip
  0% 256 Open              45% 19968 Open                86% 38400 Open               
ERRORS:
Unexpected end of archive

--
Path = /content/EdNet-KT4.zip
Type = zip
ERRORS:
Unexpected end of archive
Physical Size = 176160768

 18%      1% 669 - KT4/u362904.csv                

In [ ]:
import zipfile
import pandas as pd
import random

zip_path = "/content/EdNet-KT4.zip"

# ZIP 내부에서 바로 파일 목록 읽기
zf = zipfile.ZipFile(zip_path)
file_list = [f for f in zf.namelist() if f.endswith(".csv")]
print("총 사용자 수:", len(file_list))

# 500명 무작위 샘플링
sample_files = random.sample(file_list, 500)

user_stats = []

for f in sample_files:
    try:
        with zf.open(f) as fp:
            df = pd.read_csv(fp)
            if df.empty:
                continue

            user_id = f.split("/")[-1].replace(".csv", "")

            # 소요시간
            if "timestamp" in df.columns:
                df = df.sort_values("timestamp")
                duration = df["timestamp"].iloc[-1] - df["timestamp"].iloc[0]
            else:
                duration = None

            quit_count = (df["action_type"] == "quit").sum()
            submit_count = (df["action_type"] == "submit").sum()
            unique_q = df[df["item_id"].astype(str).str.startswith("q")]["item_id"].nunique()
            submit_ratio = submit_count / unique_q if unique_q > 0 else None

            mobile_ratio = (df["platform"] == "mobile").mean()
            web_ratio = (df["platform"] == "web").mean()

            user_stats.append({
                "user_id": user_id,
                "quit_count": quit_count,
                "duration_ms": duration,
                "mobile_ratio": mobile_ratio,
                "web_ratio": web_ratio,
                "submit_ratio": submit_ratio
            })
    except Exception as e:
        print(f"{f} 오류: {e}")

# 요약 통계
stats_df = pd.DataFrame(user_stats)
print("✅ 샘플 통계 완료:", stats_df.shape)
print(stats_df.head())


- KT4 파일로 500명 샘플링해서 사용자 로그 quit 횟수, timestamp 차이(소요시간), mobile/web 비율, submit 비율 계산함
- 500명 행동 패턴 통계 출력